In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
os.chdir('../')

In [4]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/CC-KEH/Heart-Attack-Prediction.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="CC-KEH"
os.environ["MLFLOW_TRACKING_PASSWORD"]="b0ca75eb844047a323c32834ca8cc3c7bf97c61e"

In [14]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class Model_Evaluation_Config:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [15]:
from src.Project4.utils.common import logger,read_yaml,create_directories
from src.Project4.constants import *

class Configuration_Manager:
    def __init__(self,config_file_path=CONFIG_FILE_PATH,schema_file_path=SCHEMA_FILE_PATH,params_file_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.schema = read_yaml(schema_file_path)
        self.params = read_yaml(params_file_path)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self)->Model_Evaluation_Config:
        config = self.config.model_evaluation
        schema = self.schema.TARGET_COLUMN
        params = self.params #! Fix the issue of params
        
        create_directories([config.root_dir])
        
        model_evaluation_config = Model_Evaluation_Config(root_dir=config.root_dir,
                                                          test_data_path=config.test_data_path,
                                                          model_path=config.model_path,
                                                          all_params=params,
                                                          metric_file_name=config.metric_file_name,
                                                          mlflow_uri="https://dagshub.com/CC-KEH/Heart-Attack-Prediction.mlflow",
                                                          target_column=schema.name)
        return model_evaluation_config

In [16]:
from sklearn.metrics import accuracy_score,mean_squared_error
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import joblib

In [17]:
from src.Project4.utils.common import save_json
class Model_Evaluation:
    def __init__(self,config:Model_Evaluation_Config):
        self.config = config
    
    def evaluate_metrics(self,actual,pred):
        accuracy = accuracy_score(actual,pred)
        mse = mean_squared_error(actual,pred)
        return accuracy,mse

    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        X_test = test_data.drop([self.config.target_column],axis=1)
        y_test  = test_data[[self.config.target_column]]
        
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            predictions = model.predict(X_test)
            (accuracy,mse) = self.evaluate_metrics(y_test,predictions)
            
            # Save Metrics as local
            
            scores = {"Accuracy": accuracy,"Mean Squared Error": mse}
            save_json(path=Path(self.config.metric_file_name),data=scores)
            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("Accuracy",accuracy)
            mlflow.log_metric("Mean Squared Error",mse)
            
            # Model registry does not work with file store
            if tracking_url_type_store !='file':
                mlflow.sklearn.log_model(model,"model",registered_model_name="Best Model")
            else:
                mlflow.sklearn.load_model(model,'model')
            
        

In [18]:
try:
    config = Configuration_Manager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = Model_Evaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
    
except Exception as e:
    raise e

2024-02-28 14:09:23,590 - src_logger - INFO - Yaml file:config\config.yaml loaded successfully
2024-02-28 14:09:23,594 - src_logger - INFO - Yaml file:schema.yaml loaded successfully
2024-02-28 14:09:23,600 - src_logger - INFO - Yaml file:params.yaml loaded successfully
2024-02-28 14:09:23,602 - src_logger - INFO - Created directory at: artifacts
2024-02-28 14:09:23,603 - src_logger - INFO - Created directory at: artifacts/model_evaluation
2024-02-28 14:09:24,225 - src_logger - INFO - Json file saved at: artifacts\model_evaluation\metrics.json


c:\Users\arbas\anaconda3\envs\venv\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(
Registered model 'Best Model' already exists. Creating a new version of this model...
2024/02/28 14:09:34 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: Best Model, version 2
Created version '2' of model 'Best Model'.
